If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [ ]:
!pip install jsonlines

In [ ]:
import pandas as pd
import jsonlines
import re
import nltk
import nltk
import re
import heapq
import pickle
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


## **remove links**

In [ ]:
def remove_links(text):
    # Regular expression pattern to match URLs
    url_pattern = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")

    # Remove URLs from the text
    text_without_links = re.sub(url_pattern, "", text)

    return text_without_links

## **deleting repeated characters**

In [ ]:
def delete_repeated_characters(input_text):
    pattern  = r'(.)\1+'
    out_text = re.sub(pattern, r"\1", input_text)
    return out_text

## **cleaning text**

In [ ]:
def clean_text(input_text):
    replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

## **remove vowelization**

In [ ]:
def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

## **stop words**

In [ ]:
def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") )
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    words = word_tokenize(input_text)
    filtered_words = [word for word in words if word.casefold() not in stop_words]
    filtered_text = ' '.join(filtered_words)

    return filtered_text

## **stemming**

In [ ]:
def stem_text(input_text):
    st = ISRIStemmer()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [st.stem(w) for w in tokens]
    out_text = ' '.join(out_text)
    return out_text

# **Lemitization**

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer

def lemmatize_text(input_text):
    lemmatizer = WordNetLemmatizer()
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [lemmatizer.lemmatize(w) for w in tokens]
    out_text = ' '.join(out_text)
    return out_text


# **POS**

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk import pos_tag as pos_tag_func

def pos_lemmatize_text(input_text):
    lemmatizer = WordNetLemmatizer()
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    pos_tags = pos_tag_func(tokens)

    out_text = []
    for token, pos_tag in pos_tags:
        pos_tag = get_wordnet_pos(pos_tag)  # Convert POS tag to WordNet format
        lemma = lemmatizer.lemmatize(token, pos=pos_tag)
        out_text.append(lemma)

    out_text = ' '.join(out_text)
    return out_text


def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return 'a'  # Adjective
    elif pos_tag.startswith('V'):
        return 'v'  # Verb
    elif pos_tag.startswith('N'):
        return 'n'  # Noun
    elif pos_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if no specific match


# Example usage:
text = "The cats are running quickly."
lemmatized_text = pos_lemmatize_text(text)
print(lemmatized_text)


The cat be run quickly.


In [ ]:
def preprocessing(dataset):
    preprocessed_dataset = []
    for input_text in dataset['paragraph']:
        out_text = remove_links(input_text)
        out_text = delete_repeated_characters(out_text)
        out_text = clean_text(out_text)
        out_text = delete_stopwords(out_text)
        out_text = remove_vowelization(out_text)
        out_text = pos_lemmatize_text(out_text)
        preprocessed_dataset.append(out_text)
    return preprocessed_dataset


In [ ]:
def preprocessingsumm(dataset):
    preprocessed_dataset = []
    for input_text in dataset['summary']:
        out_text = remove_links(input_text)
        out_text = delete_repeated_characters(out_text)
        out_text = clean_text(out_text)
        out_text = delete_stopwords(out_text)
        out_text = remove_vowelization(out_text)
        out_text = pos_lemmatize_text(out_text)
        preprocessed_dataset.append(out_text)
    return preprocessed_dataset

In [ ]:
! pip install datasets evaluate transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a9551a126595883ac46c58709197b10e79e25ab654c841dbdc65a5a8957dcaa4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.30.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

## Loading the dataset

In [ ]:
!pip install jsonlines

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from string import punctuation
import jsonlines

punctuation = punctuation + '\n'
from nltk.stem.isri import ISRIStemmer
def jsonl_to_dataframe(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            data.append(line)

    df = pd.DataFrame(data)
    return df

In [ ]:
import pandas as pd
dfTr = jsonl_to_dataframe(r"drive/MyDrive/labeled_validation_dataset.jsonl")
dfV = jsonl_to_dataframe(r"drive/MyDrive/labeled_validation_dataset.jsonl")
dfTe = jsonl_to_dataframe(r"drive/MyDrive/labeled_validation_dataset.jsonl")

In [ ]:
dfTr.head()

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [ ]:
from datasets import Dataset, concatenate_datasets
from datasets import DatasetDict

dataset1 = Dataset.from_pandas(dfTr)
dataset2 = Dataset.from_pandas(dfV)
dataset3 = Dataset.from_pandas(dfTe)

raw_datasets = DatasetDict({"train": dataset1, "validation": dataset2, "test": dataset3})

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset
from evaluate import load

#raw_datasets = load_dataset("xsum")
metric = load("rouge")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['example_id', 'paragraph', 'summary'],
        num_rows: 154
    })
    validation: Dataset({
        features: ['example_id', 'paragraph', 'summary'],
        num_rows: 154
    })
    test: Dataset({
        features: ['example_id', 'paragraph', 'summary'],
        num_rows: 154
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"][0]

{'example_id': 0,
 'paragraph': 'وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب عرض الكتاب الرابع ، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67 وأنها ارتاحت لاعتقادها بأن هناك وقتا طويلا وطويلا جدا قبل أن يفيق العرب من صدمة 67، وكيف أن القوات الجوية للجمهورية العربية المتحدة قد فاجأتها بعد شهر واحد من نهاية حرب 67 بهجوم جوى عنيف على مواقعها فى سيناء وكان هذا إعلانا عن بداية حرب من نوع جديد هى حرب الاستنزاف التى استمرت حتى تم وقف إطلاق النار بين الطرفين فى 8 أغسطس 1970، ثم وفاة عبدالناصر وتولى أنور السادات حكم مصر واستعداده للحرب . ويتعرض الكاتب أيضا وبصورة سريعة لفلسطين والأردن وسوريا قبل أن ينتقل إلى الكتاب الخامس عن حرب أكتوبر ، حيث يعرض الخطط والاستعدادات المصرية ثم الاستعدادات الإسرائيلية ثم يبدأ بعرض وقائع الحرب بداية من الضربة الجوية وانهيار خط بارليف واختراقه ، ويتوقف الكاتب عند يوم 8 أكتوبر ، ويقول : إن هذا اليوم كان اسوأ هزيمة فى تاريخ الجيش الإسرائيلى ثم ينتقل بنا المؤلف إلى الجبهة السورية ثم يعود ثانية إلى يوميات الحرب حتى 7 9 أكتوبر إلى 9 13 أكتوبر ثم 14 أكتوبر ، ثم يعرض للث

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,example_id,paragraph,summary
0,126,في يوم 28 فبراير عام 1922 م عاد اللورد أدموند اللنبي المندوب السامي البريطاني في مصر من العاصمة الإنجليزية لندن حاملا معه تصريح إصطلح علي تسميته تصريح 28 فبراير عام 1922 م والذى تم إصداره من جانب واحد هو الجانب البريطاني بغض النظر عن موافقة أو رفض مصر له وكان يتضمن إنهاء الحماية البريطانية علي مصر وإعلان مصر دولة مستقلة ذات سيادة مع ترك عدد 4 مسائل معلقة لم يتطرق إليها التصريح ولم يشار إليها من قريب أو بعيد في هذا التصريح وتم تجاهلها تماما وهي تأمين طرق مواصلات الإمبراطورية البريطانية والدفاع عن مصر ضد أى تدخل أجنبي وحماية الأقليات والمصالح الأجنبية في مصر وأخيرا مسألة السودان ومستقبله وكان تبرير رئيس مجلس الوزراء البريطاني حينذاك ديفيد لويد جورج لذلك أن مصر في ذلك الوقت ليس لديها حكومة رسمية قوية يمكن الإرتباط معها بمعاهدة مع بلاده وتكون قادرة علي الإلتزام بتحقيق كافة الضمانات التي تطلبها بريطانيا فيما يتعلق بتلك المسائل المعلقة وبصدور هذا التصريح قبل عبد الخالق باشا ثروت تشكيل الوزارة يوم أول مارس عام 1922 م حيث كان رافضا شغل هذا المنصب ووضع شرطين لقبوله تشكيل الوزارة وهما ضرورة رفض مشروع المعاهدة التي عرضت علي الوفد المصرى برئاسة عدلي يكن باشا رئيس الوزراء السابق في العاصمة البريطانية لندن أثناء مفاوضاته مع اللورد جورج كروزون وزير الخارجية البريطاني مع إلغاء الحماية عن مصر فلما تحقق هذان الشرطان بموجب صدور التصريح المشار إليه قبل تشكيل الوزارة وفي يوم 15 مارس عام 1922 م تم الإعلان رسميا عن إستقلال مصر وأصبح إسمها المملكة المصرية ونودى بالملك فؤاد الأول ملكا عليها ولم يعد لقبه السلطان فؤاد بل أصبح الملك فؤاد الأول وقام بإصدار مرسوم ينظم وراثة العرش للأكبر من أولاده الذكور وكان قد أنجب إبنه الأمير فاروق في ذلك الوقت فأصبح هو ولي العهد وتم إعتبار هذا اليوم عيدا قوميا ولم يبتهج المصريون بهذا التصريح نظرا لكون إستقلال مصر بهذا الشكل جاء منقوصا وقوبل بعدم الرضا وحاول عبد الخالق باشا ثروت تأكيد وتحقيق مظاهر الإستقلال فعين وكلاء وزارات مصريين بدلا من البريطانيين وأنشأ وزارة الخارجية التي تولاها بنفسه هي ووزارة الداخلية إلي جانب رئاسته لمجلس الوزراء ووافقت الحكومة البريطانية على السماح لمصر بإرسال بعثات دبلوماسية للخارج وقد بدأ ذلك في العام التالي 1923 م وكانت هذه هي المرة الأولى منذ حوالي 4 قرون أى منذ الفتح العثماني لمصر عام 1517 م التي يكون لمصر تمثيل دبلوماسي في دول العالم .,اللورد أدموند عاد ومعه تصاريح وتشمل إعلان إنهاء الحماية البريطانية على مصر وأن مصر دولة مستقلة، ومن تبريرات ديفيد لويد أن مصر لا تملك حكومة رسمية، وأن بريطانيا لا تعتمد على مصر أنها تحقق كل الضمانات التي تطلبها بريطانيا، وكان عبد الخالق يرفض أن يأخذ المنصب وقام بوضع شرطين لكي يقبل بتشكيل الوزارة، وتم الإعلان رسميا عن إستقلال مصر وصار اسمها المملكة المصرية، وكان من أول ملوك مصر الملك فؤاد، وقامت الحكومة البريطانية بالموافقة على أنها تسمح لمصر ترسل بعثات دبلوماسية للخارج وتعتبر هذه المرة الأولى منذ قرون التي يكون لمصر تمثيل دبلوماسي في دول العالم.
1,59,"أكراد العراق هم جزء مكمل للأمة الكردية التي تقطن في أرض كردستان الممتدة من لورستان إيران علما ان منذ بدء التاريخ أسسوا دولتهم الخاصة , ولكن بعد الثورة العربية الكبرى وتقسم الدول العربية , تقسمت كورديستان بموجب اتفاقية سايكس بيكو إلى 4 اجزاء , و كل جزء منها اصبح تابع لدولة اخرى , حيث يتوزعون جنوبا إلى اورمية شرقا ومن ثم إلى سيواس غربا داخل الحدود الرسمية لتركيا الحالية مرورا بشمال العراق وشمال شرق سوريا . ان الوضع السياسي والاقتصادي والاجتماعي للكرد في الجزء العراقي هو الأفضل مقارنة لوضع الكرد في الجهات الاربع . لانهم جاهدو وسعو كثيرا بعد ثورات كثيرة حصلت حصلو على اقليم كورديستان تحت نظام حكم عراقي ديمقراطي في سبعينات القرن العشرين وهذا يعود بالطبع إلى ان العراق قد تأسس اصلا من التحام الجزء الجنوبي من كردستان المنفصل حديثا من الهيمنة العثمانية الايلة للسقوط ، بالدولة الجديدة المسماة العراق التي شكلها الإنكليز بعد سايكس بيكو حيث ان الحاق الكرد بالعراق الجديد حافظ على التوازن المذهبي وشكل حجر اساس لعراق قوي قادر على الديمومة . ورغم أن العلاقة بين المركز في العراق والكرد قد مرت بحقب من الخلافات العميقة ادت إلى نشوب حروب طاحنة شرخت العلاقة المتينة التي تربط المكون الكردي مع بقية مكونات العراق .. الا انه يبدو أن الكرد يفضلون البقاء ضمن العراق الديمقراطي الفدرالي حاليا لسوء اوضاع العراقيين بسبب إيران والمالكي . مع انهم يتمنون عودة وطنهم كورديستان . وفي مقابلة مع الزعيم الكردي جلال ط

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:

metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [ ]:
fake_preds = ["مرحبا", "اهلا"]
fake_labels = ["مرحبا", "اهلا"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("مرحا اهلا بك")

{'input_ids': [0, 18935, 12, 10725, 354, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 49983, 10225, 32, 36521, 31, 12233, 1772, 9278, 19984, 251, 2], [0, 48288, 13372, 19786, 28729, 1772, 9278, 19984, 7, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them using the `text_target` parameter. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 49983, 10225, 32, 36521, 31, 12233, 1772, 9278, 19984, 251, 2], [0, 48288, 13372, 19786, 28729, 1772, 9278, 19984, 7, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    text = preprocessing(examples)
    summary = preprocessingsumm(examples)
    inputs = [prefix + doc for doc in text]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=summary, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
preprocess_function(raw_datasets['train'])

{'input_ids': [[0, 7056, 816, 14639, 48, 27145, 2094, 1198, 509, 577, 1272, 7042, 99, 480, 14216, 45475, 17, 95, 5764, 112, 22559, 30, 44, 14498, 17, 182, 9654, 4989, 10, 16431, 12, 255, 5374, 2507, 400, 16423, 2406, 675, 1541, 35259, 110, 157, 43749, 17, 289, 497, 974, 37655, 2314, 63, 20548, 624, 17, 46, 2729, 193, 1422, 12, 468, 974, 877, 295, 1450, 974, 3065, 90, 34102, 183, 5208, 113, 1338, 1283, 865, 3394, 46, 2034, 22273, 25620, 12038, 8444, 879, 131, 28162, 9, 29051, 44247, 1198, 31389, 5000, 27141, 11442, 6488, 10853, 577, 1461, 974, 3222, 2383, 24725, 22, 433, 16326, 1420, 2094, 5986, 11556, 515, 468, 21234, 1541, 36946, 948, 4553, 10410, 10, 24653, 9, 132, 7049, 1198, 107, 1917, 76, 44096, 8832, 46, 409, 399, 8779, 10853, 4825, 3924, 703, 1521, 2095, 32574, 515, 3222, 44, 1210, 3576, 13, 2655, 6578, 18653, 3166, 107, 36098, 453, 10339, 7740, 468, 7352, 367, 164, 12038, 8444, 10853, 1198, 40532, 183, 4166, 10, 30312, 376, 48854, 6582, 20928, 44, 26893, 675, 1256, 433, 25620, 

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

# hyperparameter tuning

In [ ]:
!pip install optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00


In [ ]:
import optuna
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    batch_size = 1
    model_name = model_checkpoint.split("/")[-1]
    args = Seq2SeqTrainingArguments(
        f"{model_name}-finetuned-xsum",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=1,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=True,
    )
    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    result = trainer.evaluate(eval_dataset=tokenized_datasets["validation"])

    return result["eval_loss"]


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=8)

# Get the best hyperparameters
best_learning_rate = study.best_params["learning_rate"]


[I 2023-07-05 16:08:14,731] A new study created in memory with name: no-name-4eba1a10-2b58-4e6b-8d87-d3180b478140
<ipython-input-43-d6f9ce6922e7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
Cloning https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BarthezTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than usin

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.058998,0.000000,0.000000,0.000000,0.000000,20.000000


[I 2023-07-05 16:09:57,643] Trial 0 finished with value: 3.0589983463287354 and parameters: {'learning_rate': 7.370342903183099e-05}. Best is trial 0 with value: 3.0589983463287354.
<ipython-input-43-d6f9ce6922e7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/content/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum is already a clone of https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warni

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.950338,0.000000,0.000000,0.000000,0.000000,20.000000


[I 2023-07-05 16:11:45,251] Trial 1 finished with value: 2.9503376483917236 and parameters: {'learning_rate': 2.0603673117247236e-05}. Best is trial 1 with value: 2.9503376483917236.
<ipython-input-43-d6f9ce6922e7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/content/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum is already a clone of https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warn

...

[I 2023-07-05 16:18:28,975] Trial 5 finished with value: 2.8662569522857666 and parameters: {'learning_rate': 3.328889038158605e-05}. Best is trial 5 with value: 2.8662569522857666.
<ipython-input-43-d6f9ce6922e7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/content/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum is already a clone of https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warni

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.092837,0.000000,0.000000,0.000000,0.000000,20.000000


[I 2023-07-05 16:20:05,305] Trial 6 finished with value: 3.092836856842041 and parameters: {'learning_rate': 1.7747402092450374e-05}. Best is trial 5 with value: 2.8662569522857666.
<ipython-input-43-d6f9ce6922e7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/content/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum is already a clone of https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warni

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.745963,0.000000,0.000000,0.000000,0.000000,20.000000


[I 2023-07-05 16:21:41,278] Trial 7 finished with value: 3.7459633350372314 and parameters: {'learning_rate': 5.864476310818144e-06}. Best is trial 5 with value: 2.8662569522857666.


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=3.328889038158605e-05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/content/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum is already a clone of https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.523992,0.000000,0.000000,0.000000,0.000000,20.000000


TrainOutput(global_step=10, training_loss=3.2043376922607423, metrics={'train_runtime': 20.6697, 'train_samples_per_second': 7.451, 'train_steps_per_second': 0.484, 'total_flos': 33623481065472.0, 'train_loss': 3.2043376922607423, 'epoch': 1.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/531M [00:00<?, ?B/s]

Upload file runs/Jul05_16-26-45_71d3c851f51d/events.out.tfevents.1688574412.71d3c851f51d.6613.19:   0%|       …

Upload file training_args.bin:   0%|          | 1.00/4.06k [00:00<?, ?B/s]

Upload file runs/Jul05_16-25-07_71d3c851f51d/events.out.tfevents.1688574318.71d3c851f51d.6613.18:   0%|       …

To https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum
   49b7a19..c86b82c  main -> main

   49b7a19..c86b82c  main -> main

To https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum
   c86b82c..6459ce6  main -> main

   c86b82c..6459ce6  main -> main



'https://huggingface.co/hazemOmrann14/AraBART-summ-finetuned-xsum-finetuned-xsum-finetuned-xsum/commit/c86b82cf5d21cce6faa8478044c678894c5748ab'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```